# Rough Set check functions


In [ ]:
import numpy as np
import pandas as pd

from skrough.chaos_measures import entropy
from skrough.checks import (
    check_if_approx_reduct,
    check_if_bireduct,
    check_if_consistent_table,
    check_if_functional_dependency,
    check_if_reduct,
)
from skrough.dataprep import prepare_factorized_data

## Dataset

Let's prepare a sample data set - "Play Golf Dataset".


In [ ]:
df = pd.DataFrame(
    np.array(
        [
            ["sunny", "hot", "high", "weak", "no"],
            ["sunny", "hot", "high", "strong", "no"],
            ["overcast", "hot", "high", "weak", "yes"],
            ["rain", "mild", "high", "weak", "yes"],
            ["rain", "cool", "normal", "weak", "yes"],
            ["rain", "cool", "normal", "strong", "no"],
            ["overcast", "cool", "normal", "strong", "yes"],
            ["sunny", "mild", "high", "weak", "no"],
            ["sunny", "cool", "normal", "weak", "yes"],
            ["rain", "mild", "normal", "weak", "yes"],
            ["sunny", "mild", "normal", "strong", "yes"],
            ["overcast", "mild", "high", "strong", "yes"],
            ["overcast", "hot", "normal", "weak", "yes"],
            ["rain", "mild", "high", "strong", "no"],
        ],
        dtype=object,
    ),
    columns=["Outlook", "Temperature", "Humidity", "Wind", "Play"],
)
TARGET_COLUMN = "Play"
x, x_counts, y, y_count = prepare_factorized_data(df, TARGET_COLUMN)

## Data table consistency

Let's check if the data table is consistent:

- check whole table
- check using a given subset of attributes

In [ ]:
check_if_consistent_table(x, y)

In [ ]:
# check using only first two columns
check_if_consistent_table(x[:, 0:2], y)

## Check functional dependency

In [ ]:
# check functional dependency on all objects (using default: `None`) and all attrs
# (using default: `None`)
check_if_functional_dependency(x, y)

In [ ]:
# check on all objects (using default: `None`) and on attrs `0, 2, 3`
check_if_functional_dependency(x, y, attrs=[0, 2, 3])

In [ ]:
# check on all objects (using default: `None`) and on attrs `0, 1`
check_if_functional_dependency(x, y, attrs=[0, 1])

In [ ]:
# check on objects `0, 2, 5` and on attrs `0, 1`
check_if_functional_dependency(x, y, objs=[0, 2, 5], attrs=[0, 1])

## Check reducts

For "Play Golf Dataset" there are only two reducts:

- "Outlook", "Temperature", "Humidity" - `attrs == [0, 1, 2]`
- "Outlook", "Humidity", "Wind" - `attrs == [0, 2, 3]`

In [ ]:
check_if_reduct(x, x_counts, y, y_count, attrs=[0, 2, 3])

In [ ]:
check_if_reduct(x, x_counts, y, y_count, attrs=[0, 2, 3])

In [ ]:
# too few attributes ~ no functional dependency
check_if_reduct(x, x_counts, y, y_count, attrs=[0, 1])

In [ ]:
# too many attributes ~ some of them can be removed
check_if_reduct(x, x_counts, y, y_count, attrs=[0, 1, 2, 3])

## Check approximate reducts


Check if a given subset of attributes is an approximate reduct with a given
approximation level $\varepsilon$.

See that for the specified subset of attributes and lower values of $\varepsilon$ the
answer is "no". After reaching specific larger values, the subset become good enough
to fulfill the approximation condition. However, increasing the $varepsilon$ value
even further, the subset starts to have redundant attributes (not needed to still
fulfill the approximate condition) and therefore the whole subset cannot be further
considered as an approximate reduct.

In [ ]:
attrs = [0, 3]
for eps in np.arange(0, 1, step=0.1):
    is_approx_reduct = check_if_approx_reduct(
        x, x_counts, y, y_count, attrs=attrs, chaos_fun=entropy, epsilon=eps
    )
    print(f"is approximate reduct {attrs=} for {eps=:.2} == {is_approx_reduct}")

## Check bireducts

Check if a given pair of objects and attributes subsets constitutes a decision
bireduct.

In [ ]:
df.sort_values(["Temperature", "Humidity"])

In [ ]:
check_if_bireduct(
    x, x_counts, y, y_count, objs=[0, 1, 2, 5, 6, 7, 11, 12, 13], attrs=[0]
)

In [ ]:
check_if_bireduct(x, x_counts, y, y_count, objs=[0, 1], attrs=[0])

In [ ]:
check_if_bireduct(x, x_counts, y, y_count, objs=[0, 1, 5, 7, 13], attrs=[1])

In [ ]:
# too few objects
check_if_bireduct(x, x_counts, y, y_count, objs=[7, 9, 10, 12, 13], attrs=[1, 2])

In [ ]:
check_if_bireduct(x, x_counts, y, y_count, objs=[2, 5, 7, 9, 10, 12, 13], attrs=[1, 2])

In [ ]:
check_if_bireduct(
    x,
    x_counts,
    y,
    y_count,
    objs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    attrs=[0, 2, 3],
)

In [ ]:
# all objects + all attrs - not a bireduct because some attrs are redundant
check_if_bireduct(
    x,
    x_counts,
    y,
    y_count,
    objs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    attrs=[0, 1, 2, 3],
)